# DNN Assignment
In this assignment you are working together with your teammates from the second project. You will apply your new knowledge about dense neural networks to the data from your ML project to investigate, if you can make further improvements on prediction performance. Your data is (hopefully) already cleaned and transformed (this was part of your ML project) such that you can focus fully on feeding it to your neural network. Use TensorFlow 2.x in this assignment as it makes training with real-life data much more easier with many implemented features (e.g. early-stopping, tensorboard, regularization, etc.). 

In this notebook you will learn
- how to apply a neural network to your own data using TensorFlow 2.x
- how to tune the network and monitor learning
- how to train several networks and ensemble them into a stronger model

# Module loading
Load all the necessary packages for your assignment. We give you some modules in advance, feel free to add more, if you need them.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers.experimental import preprocessing

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

print('Using TensorFlow version: %s' % tf.__version__)

Using TensorFlow version: 2.8.0


In [2]:
#!pip install -q git+https://github.com/tensorflow/docs
    
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [3]:
rseed = 42
tf.keras.backend.set_floatx('float64')
plt.rcParams['figure.figsize'] = (15, 10)

## Data loading
Load here your data from your ML project. You can use either `pandas` or `numpy` to format your data. 

In [4]:
#import Data
df=pd.read_csv("./data/air_quality_final.csv")

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

## Data Preparation
Train-Test-Split and dummy encoding (if necessary) 

In [6]:
X = df.drop('target', axis=1)
y = df['target']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")

We have 14833 observations in our dataset and 23 features
Our target vector has also 14833 values


In [7]:
X.head()

,location,time_of_day_angle,last_temp,last_precip,last_rel_humidity,last_wind_dir_x,last_wind_dir_y,last_wind_spd,last_atmos_press,mean_temp,...,mean_rel_humidity,std_rel_humidity,mean_wind_dir_x,std_wind_dir_x,mean_wind_dir_y,std_wind_dir_y,mean_wind_spd,std_wind_spd,mean_atmos_press,std_atmos_press
0,C,0.832452,19.583333,0.000,0.874583,0.823917,-0.566710,0.855000,87.768333,22.299527,...,0.758530,0.120234,0.323455,0.668548,0.041438,0.668359,0.677348,0.394041,87.752434,0.070374
1,D,4.173034,21.450000,0.561,0.797333,0.453481,-0.891266,1.019167,90.529167,24.679063,...,0.758058,0.160952,-0.061331,0.655721,0.146340,0.738142,0.899208,0.539199,90.429924,0.156000
2,A,2.509879,30.091667,0.000,0.437917,-0.340293,-0.940319,1.746667,88.415833,24.112317,...,0.712944,0.168129,-0.186449,0.724115,-0.146329,0.647675,1.038656,0.585221,88.420931,0.179481
3,A,2.536469,24.641667,0.000,0.719667,0.875622,-0.482997,1.852500,88.465833,20.845273,...,0.898326,0.114207,0.550963,0.620226,-0.246912,0.500793,1.365202,0.580259,88.376915,0.161758
4,E,1.737804,25.075000,0.000,0.773000,-0.819840,0.572592,1.301667,88.510000,22.581950,...,0.795031,0.102455,-0.384539,0.739400,0.175220,0.524133,1.133221,0.531331,88.437645,0.147604


In [ ]:
#dummy-encode the location feature
location = pd.get_dummies(X['location'], prefix='location', drop_first=True)
location.head()
# concatenate dummy-encoded locations feature to original dataframe
X = pd.concat([X, location],axis = 1)

In [ ]:
#train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=X['location'], random_state=rseed)

In [ ]:
#drop multiclass location column
X_train_loc = X_train.location
X_test_loc = X_test.location
X_train.drop('location', axis=1, inplace=True)
X_test.drop('location', axis=1, inplace=True)

In [ ]:
X_train.columns

## Training
For training you need a train/val split (hopefully you did a train/test split before (and you should use the same as in your ML project to make results comparable). 

In [ ]:
valsize = 0.8
N_VAL = len(X_train)*valsize
N_TRAIN = len(X_train)-N_VAL
BATCH_SIZE = 64
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
EPOCHS = 100

In [ ]:
#Learning Rate Decay
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.01,
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False)

### Build, compile and fit your model
To become fast at retraining your (different) models it is good practice to define a function that gets fed by a model, its name, an optimizer to use and the number of epochs you want the model to be trained. 

If your model trains for many epochs you will receive a lot of logging from TensorFlow. To reduce the logging noise you can use a callback (provided by the `tensorflow_docs` module we installed and imported for you) named `EpochDots()` that simply prints a `.` for each epoch and a full set of metrics after a number of epochs have been trained. 

If you want to produce logs for using Tensorboard you also need to include the `callbacks.Tensorboard()`.

In [ ]:
#def get_callbacks(name):
#    return tf.keras.callbacks.TensorBoard(run_logdir+name, histogram_freq=1)

You can implement your callbacks in the `model.fit()` method below.

In [ ]:
# Define optimizer used for modelling
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, name='Adam')

In [ ]:
#def model_compile_and_fit(model, name, optimizer=None, max_epochs=30):
    # Get optimizer
    
    # model.compile
    
    # model.fit
    
    # return results

In [ ]:
#def model_compile_and_fit(model, name, optimizer=Adam, max_epochs=30,loss='mae',metrics=['mse']):
#    model.compile(optimizer=optimizer,
#    loss=loss
#    metrics=metrics)

#### Build your model
You can build your model by using `tf.keras.Sequential()` that helps you to sequentially define your different layers from input to output. 

In [ ]:
normalizer = preprocessing.Normalization()
normalizer.adapt()

In [ ]:
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])    

#### Train your model
Train your model by using your `model_compile_and_fit()` function you defined above.

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mae',
    metrics='mse')

In [ ]:
model.fit(
    df_train['Horsepower'], y_train,
    epochs=50,
    # suppress logging (if you want to see the output for the different epochs set the value to 1)
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2,
    # Store information for TensorBoard
    callbacks=get_callbacks("horsepower_model"))

In [ ]:
#your_history = model_compile_and_fit(your_model, ....)

#### Evaluate your model training
TensorFlow offers now (this was more cumbersome before) a simple history plotter that you can use to plot training histories and see how the model performed on training and validation data set.

In [ ]:
# history_plotter = tfdocs.plots.HistoryPlotter(metric = 'your_metric', smoothing_std=10)
# history_plotter.plot(your_history)

## Model tuning
You might have no luck with your first model (most surely you did not). In this section you will apply methods you know to tune your model's performance. An obvious way of course is to change your model's architecture (removing or adding layers or layer dimensions, changing activation functions). 

However, after this you might still be able to detect some overfitting and there are some more methods you can apply to improve your neural network. Some of them are regularization, learning rate decay, early stopping, or dropout. 

If you want to add regularization you can apply directly layer-wise L2- or L1-regularization by using a layer's `kernel_regularization` argument and an appropriate regularizer from the [`tensorflow.keras.regularizers`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) module we imported for you.  

__Optimizer schedules__<br>
Quite often your optimizer does not run efficiently through the loss function surface. Remember that theory ensures a convergence of mini-batch SGD if and only if the learing rate decreases sufficiently fast. A way to apply this to your model training is to use a learning rate scheduler (learning rate decay) that reduces the learning rate over the number of update steps. The [`tf.keras.optimizers.schedules`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules) module offers you some approaches to do that. 

Note that to apply this to your `model_compile_and_fit()` function you defined above you need to implement the learning rate schedule either in there or with a helper function that your function calls inside. 

If you want to visualize different schedulers you can define them and call them on a range of values and plot them in a line plot. 

__Early stopping__<br>
Earyl stopping is a procedure that enables you to stop your training earlier than defined by your `max_epochs` argument. It is used in practices to 
1. determine the optimal parameter vector by monitoring the validation error closely (if it rises again too much stuck with the best parameters found until then) and
2. to save expensive resources (either in terms of monetary costs or ecological costs).

To implement early stopping in TensorFlow the `tf.keras` module offers you a `callback` named [`tf.keras.callback.EarlyStopping()`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) that monitors for you a certain metric (it makes sense here to use a validation metric) and to stop training after a certain number of epochs with no improvement or by defining a certain `min_delta` that defines a minimum value of improvement - if below the callback stops your training. 

You can add this callback simply to the callbacks defined in your `get_callbacks()` function you defined above.

__Dropout__<br>
Dropout was one of the important developments in regularization for neural networks. It was developed by Geoffrey Hinton and his team at Toronto University. 

Dropout can be applied to each layer in your network and is implemented in `tf.keras` by an own layer named `Dropout()` awaiting a dropout rate set by you. So to introduce dropout you have to rework your model design.  

Make use of your knowledge and apply tuning techniques to improve your network. 

In [ ]:
#===========#
# YOUR CODE #
#===========#

## Model ensembling
You have learned that models can be ensembled. What is possible in `scikit-learn` is also possible in TensorFlow, just a little different as it is relying on its computation graph. However, any model is callable like a `layer` by invoking it on either an `Input` or on the output of another layer. Furthermore, you can also stack outputs together.

To produce an ensemble you can define a couple of models, than use their predictions as inputs for another model and produce a final output (using `keras.Model(input, output)`). But you can also start simple and use the mean predictions over all models and then compute the `argmax()` to assign them to a class in classification (via using `layers.average([model1_preds,model2_preds,...])`). You will be surprised how well this works. 

Now implement your own ensemble to improve your work even a little more and to have something more to polish up your ML project on `GitHub` ;) 

In [ ]:
#===========#
# YOUR CODE #
#===========#